In [3]:
import torch
!pip install transformers

     |████████████████████████████████| 3.8 MB 32.3 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 64.9 MB/s 
     |████████████████████████████████| 6.5 MB 62.5 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to('cuda')

In [36]:
import torch.nn as nn
import numpy as np
import re
softy = nn.Softmax(dim = -1)

In [54]:
def get_sentiment(text) :
  input_ids1 = tokenizer(text, truncation=True,padding=True, max_length=128, return_tensors='pt').input_ids.to('cuda')
  with torch.no_grad() :
    output = model(input_ids1)['logits']
  probs = softy(output).cpu().numpy()
  # infer_dict = {'positive' : probs[0],'negative' : probs[1], 'neutral' : probs[2]}
  return probs


In [75]:
def process_text(document):
  document = re.sub('[^\.A-Za-z0-9]+', ' ', document)
  # document = document.lower()   
  return document

In [76]:
def get_output(text) :
  processed_text = process_text(text)
  sents = list(processed_text.split('.'))
  filtered_sents = [s.strip() for s in sents if len(s.split()) > 4]
  sentiment = get_sentiment(filtered_sents)
  mean_vals = np.mean(sentiment,axis = 0).tolist()
  infer_dict = {'positive' : mean_vals[0],'negative' : mean_vals[1],'neutral' : mean_vals[2]}
  return infer_dict
  


In [77]:
get_output('Microsoft is a great company')

{'negative': 0.011568653397262096,
 'neutral': 0.6484085321426392,
 'positive': 0.34002286195755005}